In [263]:
import numpy as np
import math
import residuals as r
np.set_printoptions(threshold=40)
np.set_printoptions(suppress=True)

In [379]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

factor10 = math.sqrt((25*60)/10.0) 
factor80 = math.sqrt((25*60)/80.0) 



def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1 / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))

def magnitude(x):
    var = float(x)
    return int(math.floor(math.log10(var)))

def jitter_ratio(resids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all) as entire observation period
    x=r.read_residuals(filename=resids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid_all = x.prefit_sec[condition]
    Errors_all = x.uncertainty[condition]
    std_all = weighted_stddev(Resid_all, Errors_all)
    
    ##============
    
    #Define sigma_dump (or std_all) as individual dumps within observation period
    
    #This isolates the numbers for the different dump times
    rounded = np.round(np.array([x.bary_TOA[condition] - 0.00005]), 4)
    dumptimes=[]
    for i in rounded.tolist()[0]: 
        if i not in dumptimes:
            dumptimes.append(i)

    #This calculates a list of sigma_dump values and averages those
    str_dump_devs=[]
    for i in dumptimes:
        timeindex = (rounded == i)[0]
        Resid = Resid_all[timeindex]
        Errors = Errors_all[timeindex]
        single_std_dump = weighted_stddev(Resid, Errors)
        str_dump_devs.append(single_std_dump)
    
    #to convert str_dump_devs out of scientific notation
    dump_devs=[]
    for i in str_dump_devs:
        dump_devs.append('{0:.20f}'.format(i))
        
    #prints the magnitude of each sigma_dump in a list and finds average magnitude
    magarray=[]
    for i in dump_devs:
        magarray.append(magnitude(i))
    averagemag = round(np.mean(magarray))
    
    arg=averagemag+1.5
    makemask = np.ma.masked_where(np.array(magarray) > arg, np.array(magarray))
    
    std_dump=np.mean(str_dump_devs)
    masked_std_dump = np.mean(np.ma.masked_where(np.ma.getmask(makemask), str_dump_devs))
        
#     for i in dumptimes:
#         timeindex = (rounded == i)[0]
#         Errors = Errors_all[timeindex]
#         print Errors
    
    #Setup jitter ratio
    ratio = std_all / std_dump
    masked_ratio = std_all / masked_std_dump
    
    print titlearg
    print "sigma_all = " + str(std_all)
    print "average sigma_dump = " + str(std_dump)
#     print "every sigma_dump = " + str(dump_devs)
    print "jitter ratio = " + str(ratio) 
    print "masked jitter ratio = " + str(masked_ratio) 

In [92]:
#Checking order of operations

print 25*60
print 1500/10
print math.sqrt(150)
print math.sqrt((25*60)/10)

print math.sqrt((25*60)/80)

#yes, order of operations works

1500
150
12.2474487139
12.2474487139
4.24264068712


In [249]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals')

[  9.22596685e-08   8.99622660e-08   7.84792957e-08   6.01280461e-08
   4.75406195e-08   4.29740921e-08   2.48466751e-08   1.71108451e-08
   3.16053786e-08   4.98264696e-08   8.19233902e-08   1.29040396e-07]
[  1.03749738e-07   1.00302237e-07   8.99622660e-08   7.38884627e-08
   6.12738609e-08   5.55469309e-08   3.84165110e-08   2.48466751e-08
   1.92942759e-08   3.50062984e-08   6.81524197e-08   1.10645768e-07]
[  9.45572945e-08   9.11109382e-08   8.07752742e-08   6.24198666e-08
   4.98264696e-08   4.52564067e-08   2.93448573e-08   1.71108451e-08
   2.26135620e-08   4.06939953e-08   7.15936804e-08   1.18692602e-07]
[  9.80041185e-08   9.68551283e-08   8.65166207e-08   6.92993768e-08
   5.66918621e-08   5.09699398e-08   3.38714875e-08   2.03957720e-08
   2.15024545e-08   3.61421331e-08   7.04464658e-08   1.17542969e-07]
[  9.68551283e-08   9.45572945e-08   8.42198584e-08   6.70056012e-08
   5.32578533e-08   4.98264696e-08   3.16053786e-08   1.92942759e-08
   2.15024545e-08   3.61421331

In [240]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals')

J1713+0747, MJD 56380, 10 second subintervals
sigma_all = 5.82671845721e-07
average sigma_dump = 7.10113781891e-08
every sigma_dump = [7.652615150725228e-08, 5.902018619731153e-08, 4.20331293144798e-08, 4.4507325877169695e-08, 6.200782302091449e-08, 5.045884132766692e-08, 6.440799119689885e-08, 9.036416007993344e-08, 4.319425564252257e-08, 5.6341775343032575e-08, 1.0336494082401902e-07, 6.560570986872777e-08, 4.612109491866342e-08, 3.6932430266857896e-08, 6.376835014042846e-08, 3.954673052089589e-08, 7.169240320710303e-08, 6.687428619769905e-08, 7.808116467353697e-08, 4.9081806645279844e-08, 5.786274090057513e-08, 4.2919321130398024e-08, 4.2327435350973444e-08, 4.9783182135463393e-08, 9.599978527397674e-08, 3.589504398660005e-08, 6.291261822976606e-08, 4.122706551922135e-08, 4.8493116939493145e-08, 4.7408930563566425e-08, 1.0439494154516478e-07, 4.643743321492075e-08, 4.047201991324815e-08, 1.1671209618338409e-07, 4.341256861407862e-08, 4.112483917678351e-08, 7.097321634576283e-08, 3.0

In [241]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals')

J1713+0747, MJD 56201, 80 second subintervals
sigma_all = 1.73501235065e-07
average sigma_dump = 5.08948679605e-08
every sigma_dump = [3.711685588402716e-08, 5.886886596802255e-08, 6.111023139470437e-08, 4.4051766062268014e-08, 3.4740437800597703e-08, 7.906660416205554e-08, 3.7411318845851984e-08, 6.097134425408946e-08, 5.171443034351814e-08, 2.905352748851564e-08, 2.876442120807968e-08, 5.5729845317009185e-08, 6.806552366376539e-08, 6.525415973583415e-08, 4.668679450568369e-08, 4.686054204702965e-08, 5.974608664736754e-08]
jitter ratio = 3.40901238215


In [242]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals')

J1713+0747, MJD 56201, 10 second subintervals
sigma_all = 5.43224438005e-07
average sigma_dump = 8.13611454109e-08
every sigma_dump = [6.50876676132087e-08, 6.46745944730762e-08, 7.865590907570538e-08, 8.928395855320836e-08, 6.333967301326343e-08, 5.637596792232031e-08, 6.71543982888649e-08, 1.076399428397337e-07, 1.0054366410120812e-07, 8.951651947992012e-08, 6.869809525821204e-08, 1.1919947814919073e-07, 4.9928606158072935e-08, 4.850495966392457e-08, 6.791470194640791e-08, 6.29092352730041e-08, 8.472060024126327e-08, 6.309083187347114e-08, 9.954005809369425e-08, 9.797198870898249e-08, 7.234262916436704e-08, 4.789747873451385e-08, 3.725214457545979e-08, 1.5225451598479727e-07, 6.407553843519054e-08, 5.373267496803205e-08, 4.9397312258131903e-08, 1.0261281750165839e-07, 5.710729415846082e-08, 8.441147862045343e-08, 1.0995768474080987e-07, 7.18878328889364e-08, 8.009506922008883e-08, 7.665686989310976e-08, 5.7302920066441385e-08, 1.270308325059373e-07, 5.6241493439914266e-08, 8.88512345

In [243]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals')

B1937+21, MJD 56465, 80 second subintervals
sigma_all = 4.68059107539e-08
average sigma_dump = 4.44210286622e-08
every sigma_dump = [4.784586452107492e-08, 4.505159462070724e-08, 5.9094750149760064e-08, 5.295389596241171e-08, 4.084974443877081e-08, 4.46180326943326e-08, 4.0991032666644566e-08, 4.139608337521056e-08, 4.242850805335143e-08, 4.793753792815118e-08, 3.978944909197951e-08, 5.008212876128583e-08, 3.0705840016888486e-08, 3.42422653535736e-08, 4.653979528926001e-08, 4.620993567211042e-08]
jitter ratio = 1.05368813293


In [244]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals')

B1937+21, MJD 56465, 10 second subintervals
sigma_all = 7.05138954322e-08
average sigma_dump = 4.92378535069e-08
every sigma_dump = [2.8193967243388876e-08, 4.181912386218402e-08, 3.927135274263921e-08, 4.3635933089685625e-08, 5.736217982072503e-08, 6.823709629846934e-08, 6.546886553169743e-08, 4.881912651899564e-08, 5.1116851798278415e-08, 3.2734029173032334e-08, 4.3524303418461514e-08, 5.471712339810651e-08, 4.884663548611178e-08, 4.102064558219938e-08, 5.5586935578483576e-08, 6.681067727755345e-08, 4.914523576775492e-08, 4.988554945451243e-08, 6.553842166516284e-08, 5.2866352114191494e-08, 4.880020726208761e-08, 6.286421577679552e-08, 6.507254266203777e-08, 5.614643368961612e-08, 6.139045875954143e-08, 4.344262892395188e-08, 5.385069958695083e-08, 2.9283096838726422e-08, 6.371782201107528e-08, 5.617585481999904e-08, 4.9221492114633746e-08, 4.3615483267505474e-08, 5.834659024770328e-08, 3.714998602474739e-08, 4.059258298342007e-08, 4.409173086328776e-08, 3.6250807443132565e-08, 3.569

In [380]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_80F8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 80 second subintervals')

J1022+1001, MJD 57208, 80 second subintervals
sigma_all = 4.6866049668e-05
average sigma_dump = 0.000296708047933
jitter ratio = 0.157953415806
masked jitter ratio = 3.33695161071


In [381]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_NTF8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 10 second subintervals')

J1022+1001, MJD 57208, 10 second subintervals
sigma_all = 0.000104350827388
average sigma_dump = 0.000345928859937
jitter ratio = 0.301654008881
masked jitter ratio = 2.1828169481


In [281]:
format?